In [1]:
import tarfile
import zipfile
import shutil
import random
import os
import cv2
import pathlib
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.random import seed
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
from tensorflow.keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet101

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
random_seed = 42

seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_global_policy(policy)
#print('Compute dtype: %s' % policy.compute_dtype)
#print('Variable dtype: %s' % policy.variable_dtype)

In [3]:
drive.mount('/content/gdrive', force_remount=True)
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/val.tar'
with zipfile.ZipFile("/content/gdrive/MyDrive/Progetto_VIPM/dataset/val_degraded.zip","r") as zip_ref:
    zip_ref.extractall()

Output streaming troncato alle ultime 5000 righe.
./val_set/val_003453.jpg
./val_set/val_005719.jpg
./val_set/val_001622.jpg
./val_set/val_002497.jpg
./val_set/val_011151.jpg
./val_set/val_007639.jpg
./val_set/val_005917.jpg
./val_set/val_010592.jpg
./val_set/val_001806.jpg
./val_set/val_005160.jpg
./val_set/val_011428.jpg
./val_set/val_006375.jpg
./val_set/val_011387.jpg
./val_set/val_007022.jpg
./val_set/val_006082.jpg
./val_set/val_002553.jpg
./val_set/val_006272.jpg
./val_set/val_002359.jpg
./val_set/val_007099.jpg
./val_set/val_006264.jpg
./val_set/val_005217.jpg
./val_set/val_008900.jpg
./val_set/val_005329.jpg
./val_set/val_009168.jpg
./val_set/val_004077.jpg
./val_set/val_010864.jpg
./val_set/val_007423.jpg
./val_set/val_006787.jpg
./val_set/val_001671.jpg
./val_set/val_002147.jpg
./val_set/val_009798.jpg
./val_set/val_011954.jpg
./val_set/val_005692.jpg
./val_set/val_005568.jpg
./val_set/val_002920.jpg
./val_set/val_005490.jpg
./val_set/val_005175.jpg
./val_set/val_005083.jpg


In [4]:
csv_test_clean_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_clean_file.columns = ['filename', 'label']
parent_dir = "val_set/"
labels = csv_test_clean_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_clean_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
val_set/16/val_005256.jpg
val_set/119/val_005257.jpg
val_set/68/val_005258.jpg
val_set/130/val_005259.jpg
val_set/43/val_005260.jpg
val_set/197/val_005261.jpg
val_set/101/val_005262.jpg
val_set/55/val_005263.jpg
val_set/62/val_005264.jpg
val_set/49/val_005265.jpg
val_set/17/val_005266.jpg
val_set/204/val_005267.jpg
val_set/181/val_005268.jpg
val_set/122/val_005269.jpg
val_set/170/val_005270.jpg
val_set/66/val_005271.jpg
val_set/48/val_005272.jpg
val_set/96/val_005273.jpg
val_set/86/val_005274.jpg
val_set/144/val_005275.jpg
val_set/99/val_005276.jpg
val_set/4/val_005277.jpg
val_set/30/val_005278.jpg
val_set/177/val_005279.jpg
val_set/182/val_005280.jpg
val_set/201/val_005281.jpg
val_set/168/val_005282.jpg
val_set/187/val_005283.jpg
val_set/130/val_005284.jpg
val_set/30/val_005285.jpg
val_set/145/val_005286.jpg
val_set/153/val_005287.jpg
val_set/180/val_005288.jpg
val_set/160/val_005289.jpg
val_set/37/val_005290.jpg
val_set/52/val_005291.

In [5]:
csv_test_deg_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_deg_file.columns = ['filename', 'label']
parent_dir = "val_set_degraded/"
labels = csv_test_deg_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_deg_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
val_set_degraded/16/val_005256.jpg
val_set_degraded/119/val_005257.jpg
val_set_degraded/68/val_005258.jpg
val_set_degraded/130/val_005259.jpg
val_set_degraded/43/val_005260.jpg
val_set_degraded/197/val_005261.jpg
val_set_degraded/101/val_005262.jpg
val_set_degraded/55/val_005263.jpg
val_set_degraded/62/val_005264.jpg
val_set_degraded/49/val_005265.jpg
val_set_degraded/17/val_005266.jpg
val_set_degraded/204/val_005267.jpg
val_set_degraded/181/val_005268.jpg
val_set_degraded/122/val_005269.jpg
val_set_degraded/170/val_005270.jpg
val_set_degraded/66/val_005271.jpg
val_set_degraded/48/val_005272.jpg
val_set_degraded/96/val_005273.jpg
val_set_degraded/86/val_005274.jpg
val_set_degraded/144/val_005275.jpg
val_set_degraded/99/val_005276.jpg
val_set_degraded/4/val_005277.jpg
val_set_degraded/30/val_005278.jpg
val_set_degraded/177/val_005279.jpg
val_set_degraded/182/val_005280.jpg
val_set_degraded/201/val_005281.jpg
val_set_degraded/168/val_0052

In [6]:
test_clean_data_dir = pathlib.Path('val_set/')
#shuffle_value = True
shuffle_value = False
batch_size = 128

test_clean_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_clean_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value,
    seed=random_seed)

Found 11993 files belonging to 251 classes.


In [7]:
test_deg_data_dir = pathlib.Path('val_set_degraded/')
#shuffle_value = True
shuffle_value = False
batch_size = 128

test_deg_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_deg_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value,
    seed=random_seed)

Found 11993 files belonging to 251 classes.


In [8]:
def preprocess(images, labels):
  return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels

#train_ds = train_ds.map(preprocess)
#val_ds = val_ds.map(preprocess)
test_clean_ds = test_clean_ds.map(preprocess)
test_deg_ds = test_deg_ds.map(preprocess)

# Importazione modello

In [9]:
checkpoint_filepath = '/content/gdrive/MyDrive/Progetto_VIPM/models/best_model_mobilenetv2_augmented.h5'
loaded_model = keras.models.load_model(checkpoint_filepath)
#loaded_model.summary()

In [10]:
lista_classi = np.genfromtxt("/content/gdrive/MyDrive/Progetto_VIPM/annot/class_list.txt", dtype=str)

In [11]:
from keras.utils import dataset_utils

def get_test_clean_labels_and_paths():
  # questa è la funzione usata da keras per creare il dataset
  file_paths, labels, class_names = dataset_utils.index_directory(directory="val_set/", 
                                                                  labels="inferred", 
                                                                  formats="jpg",
                                                                  shuffle=False,
                                                                  seed=42)
  return labels, file_paths

def get_test_deg_labels_and_paths():
  # questa è la funzione usata da keras per creare il dataset
  file_paths, labels, class_names = dataset_utils.index_directory(directory="val_set_degraded/", 
                                                                  labels="inferred", 
                                                                  formats="jpg",
                                                                  shuffle=False,
                                                                  seed=42)
  return labels, file_paths

In [12]:
y_test_clean_pred = loaded_model.predict(test_clean_ds)

94/94 [==============================] - 30s 234ms/step


In [13]:
y_test_clean_labels, test_clean_paths = get_test_clean_labels_and_paths()

Found 11993 files belonging to 251 classes.


In [14]:
y_test_clean_labels

array([  0,   0,   0, ..., 250, 250, 250], dtype=int32)

In [15]:
y_test_clean_pred

array([[7.68503606e-01, 4.02415981e-06, 1.32153446e-06, ...,
        4.61739891e-09, 1.83680839e-08, 4.49790718e-07],
       [9.99995232e-01, 4.23532515e-10, 1.32618236e-10, ...,
        5.31961509e-15, 2.47171617e-10, 2.07164716e-13],
       [2.25008383e-01, 1.45940983e-03, 4.90187481e-03, ...,
        4.05038054e-06, 5.71658136e-04, 4.60463662e-06],
       ...,
       [1.31424835e-08, 3.82756934e-06, 2.84298178e-04, ...,
        2.22665381e-06, 3.89217079e-04, 2.29967594e-01],
       [3.99617812e-13, 1.05371628e-05, 4.10882611e-04, ...,
        1.00284349e-04, 3.20177718e-10, 8.40984523e-01],
       [3.00328480e-07, 2.01545463e-05, 1.70770870e-03, ...,
        2.88450906e-07, 1.73492800e-03, 1.68077380e-03]], dtype=float32)

In [16]:
y_test_clean_pred.argmax(1)

array([  0,   0,   0, ..., 250, 250,   3])

In [17]:
cm = multilabel_confusion_matrix(y_test_clean_labels, y_test_clean_pred.argmax(1))
cm

array([[[11932,     6],
        [   10,    45]],

       [[11916,    16],
        [   12,    49]],

       [[11925,    20],
        [   25,    23]],

       ...,

       [[11916,    16],
        [   17,    44]],

       [[11903,    32],
        [   36,    22]],

       [[11965,     7],
        [   12,     9]]])

In [18]:
print(classification_report(y_test_clean_labels, y_test_clean_pred.argmax(1), zero_division=0))

              precision    recall  f1-score   support

           0       0.88      0.82      0.85        55
           1       0.75      0.80      0.78        61
           2       0.53      0.48      0.51        48
           3       0.64      0.77      0.70        47
           4       0.82      0.75      0.78        36
           5       0.43      0.46      0.44        46
           6       0.54      0.48      0.51        42
           7       0.59      0.57      0.58        40
           8       0.68      0.71      0.69        51
           9       0.44      0.64      0.52        45
          10       0.56      0.60      0.58        48
          11       0.41      0.50      0.45        36
          12       0.82      0.66      0.73        35
          13       0.47      0.55      0.51        47
          14       0.61      0.55      0.58        55
          15       0.81      0.70      0.75        60
          16       0.61      0.65      0.63        54
          17       0.62    

In [19]:
idx = np.argsort(y_test_clean_pred, axis=1)[:,-3:]
idx

array([[  4, 105,   0],
       [ 73, 139,   0],
       [ 69,  95,   0],
       ...,
       [151, 110, 250],
       [178, 151, 250],
       [227, 140,   3]])

In [20]:
y_true = y_test_clean_labels
y_true

array([  0,   0,   0, ..., 250, 250, 250], dtype=int32)

In [21]:
#Trovo la prima immagine misclassificata
index_pred = 0
class_true = -1
class_predicted = []
valid = False
for i in range(len(idx)):
  if idx[i,0] != y_true[i] and idx[i,1] != y_true[i] and idx[i,2] != y_true[i]:
    print(idx[i])
    print(y_true[i])
    index_pred = i
    class_true = y_true[i]
    class_predicted = idx[i]
    break

[246  73 186]
0


In [22]:
print("Immagine classificata non correttamente: ", test_clean_paths[index_pred])
print("Classi predette: ", class_predicted)
print("Classe corretta: ", class_true)

Immagine classificata non correttamente:  val_set/0/val_003487.jpg
Classi predette:  [246  73 186]
Classe corretta:  0


In [23]:
y_test_deg_pred = loaded_model.predict(test_deg_ds)

94/94 [==============================] - 22s 235ms/step


In [24]:
y_test_deg_labels, test_deg_paths = get_test_deg_labels_and_paths()

Found 11993 files belonging to 251 classes.


In [25]:
cm = multilabel_confusion_matrix(y_test_deg_labels, y_test_deg_pred.argmax(1))
cm

array([[[11926,    12],
        [   17,    38]],

       [[11907,    25],
        [   22,    39]],

       [[11913,    32],
        [   34,    14]],

       ...,

       [[11916,    16],
        [   26,    35]],

       [[11893,    42],
        [   41,    17]],

       [[11966,     6],
        [   18,     3]]])

In [26]:
print(classification_report(y_test_deg_labels, y_test_deg_pred.argmax(1), zero_division=0))

              precision    recall  f1-score   support

           0       0.76      0.69      0.72        55
           1       0.61      0.64      0.62        61
           2       0.30      0.29      0.30        48
           3       0.55      0.77      0.64        47
           4       0.55      0.61      0.58        36
           5       0.35      0.17      0.23        46
           6       0.37      0.38      0.38        42
           7       0.30      0.47      0.37        40
           8       0.65      0.65      0.65        51
           9       0.31      0.40      0.35        45
          10       0.44      0.52      0.48        48
          11       0.48      0.44      0.46        36
          12       0.49      0.49      0.49        35
          13       0.43      0.53      0.48        47
          14       0.55      0.42      0.47        55
          15       0.55      0.45      0.50        60
          16       0.45      0.46      0.45        54
          17       0.50    

In [27]:
idx = np.argsort(y_test_deg_pred, axis=1)[:,-3:]
idx

array([[105, 192,   4],
       [ 64, 105,   0],
       [155, 101,  95],
       ...,
       [120, 111, 202],
       [240,  36, 215],
       [227, 178,   3]])

In [28]:
y_true = y_test_deg_labels
y_true

array([  0,   0,   0, ..., 250, 250, 250], dtype=int32)

In [29]:
#Trovo la prima immagine misclassificata
index_pred = 0
class_true = -1
class_predicted = []
valid = False
for i in range(len(idx)):
  if idx[i,0] != y_true[i] and idx[i,1] != y_true[i] and idx[i,2] != y_true[i]:
    print(idx[i])
    print(y_true[i])
    index_pred = i
    class_true = y_true[i]
    class_predicted = idx[i]
    break

[105 192   4]
0


In [30]:
print("Immagine classificata non correttamente: ", test_deg_paths[index_pred])
print("Classi predette: ", class_predicted)
print("Classe corretta: ", class_true)

Immagine classificata non correttamente:  val_set_degraded/0/val_000013.jpg
Classi predette:  [105 192   4]
Classe corretta:  0
